# Tokenizer Class

> Implementation of the `Tokenizer` class, which encapsulates the BPE vocabulary and merge rules to provide encoding and decoding functionalities.

In [ ]:
#| default_exp bpe_tokenizer_class

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import regex as re
import json
import base64
from typing import List, Dict, Tuple, Optional, Iterable, Iterator, Any, Pattern
import os # For temp file creation in example
import tempfile # For temp file creation in example

# Attempt to import from local utils, will be available after nbdev_export
try:
    from .utils import load_tokenizer_components, save_tokenizer_components
except ImportError:
    # Fallback for direct notebook execution or if package not fully installed
    try:
        from bpe_tokenizer_scratch.utils import load_tokenizer_components, save_tokenizer_components
    except ImportError:
        # Define directly if utils not available (e.g. for isolated testing of this notebook cell)
        # This is not ideal for production but helps in iterative development of notebooks
        print("Warning: 'utils' module not found, defining load/save_tokenizer_components locally for Tokenizer.from_files and testing.")
        def save_tokenizer_components(vocab: Dict[int, bytes], merges: List[Tuple[bytes, bytes]], vocab_filepath: str, merges_filepath: str):
            vocab_to_save = {str(k): base64.b64encode(v).decode('ascii') for k, v in vocab.items()}
            with open(vocab_filepath, 'w', encoding='utf-8') as f: json.dump(vocab_to_save, f, indent=2)
            merges_to_save = [(base64.b64encode(p1).decode('ascii'), base64.b64encode(p2).decode('ascii')) for p1, p2 in merges]
            with open(merges_filepath, 'w', encoding='utf-8') as f: json.dump(merges_to_save, f, indent=2)

        def load_tokenizer_components(vocab_filepath: str, merges_filepath: str) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
            with open(vocab_filepath, 'r', encoding='utf-8') as f: vocab_json = json.load(f)
            loaded_vocab = {int(k): base64.b64decode(v_str) for k, v_str in vocab_json.items()}
            with open(merges_filepath, 'r', encoding='utf-8') as f: merges_json = json.load(f)
            loaded_merges = [(base64.b64decode(p1_str), base64.b64decode(p2_str)) for p1_str, p2_str in merges_json]
            return loaded_vocab, loaded_merges

In [ ]:
#| export
class Tokenizer:
    """ A BPE Tokenizer that can encode text into token IDs and decode token IDs back to text.
    
    The Tokenizer is initialized with a vocabulary (mapping IDs to byte sequences) 
    and a list of merge rules (ordered pairs of byte sequences that were merged during training).
    It also handles special tokens.
    """
    PAT_REGEX: Pattern = re.compile(r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
    
    def __init__(self, 
                 vocab: Dict[int, bytes], 
                 merges: List[Tuple[bytes, bytes]], 
                 special_tokens: Optional[List[str]] = None):
        """Initializes the Tokenizer.

        Args:
            vocab (Dict[int, bytes]): A dictionary mapping token IDs (int) to their byte representations (bytes).
                                     This vocab should ideally come from `train_bpe` and include base bytes (0-255)
                                     and any initial special tokens passed to `train_bpe`.
            merges (List[Tuple[bytes, bytes]]): An ordered list of BPE merge rules. Each rule is a tuple of two
                                                byte sequences that were merged. The order determines merge priority.
            special_tokens (Optional[List[str]]): A list of special token strings (e.g., "<|endoftext|>"). 
                                                    These tokens are handled distinctly during encoding/decoding.
                                                    If a special token's byte representation is already in `vocab`,
                                                    its existing ID is used. Otherwise, a new ID is assigned and 
                                                    it's added to the vocabulary.
        """
        self.vocab: Dict[int, bytes] = vocab.copy() 
        self.merges_map: Dict[Tuple[bytes, bytes], int] = {pair: i for i, pair in enumerate(merges)} 

        self.special_tokens_list: List[str] = []
        if special_tokens is not None:
            seen_st = set()
            for st in special_tokens:
                if st not in seen_st:
                    self.special_tokens_list.append(st)
                    seen_st.add(st)
            
        self.special_token_to_id: Dict[str, int] = {} 
        self.id_to_special_token: Dict[int, str] = {} 

        _temp_bytes_to_id: Dict[bytes, int] = {b: i for i, b in sorted(self.vocab.items(), key=lambda item: item[0])}
        
        max_internal_id = -1
        if self.vocab: 
            max_internal_id = max((k for k in self.vocab.keys() if isinstance(k, int)), default=-1)
        
        current_max_id_for_new_specials = max_internal_id

        for st_text in self.special_tokens_list:
            st_bytes = st_text.encode("utf-8")
            if st_bytes in _temp_bytes_to_id:
                existing_id = _temp_bytes_to_id[st_bytes]
                self.special_token_to_id[st_text] = existing_id
                self.id_to_special_token[existing_id] = st_text
                self.vocab[existing_id] = st_bytes 
            else:
                current_max_id_for_new_specials += 1
                new_id = current_max_id_for_new_specials
                
                self.vocab[new_id] = st_bytes      
                _temp_bytes_to_id[st_bytes] = new_id 
                
                self.special_token_to_id[st_text] = new_id
                self.id_to_special_token[new_id] = st_text
        
        self.bytes_to_id: Dict[bytes, int] = {b: i for i, b in self.vocab.items()}
        
        if self.special_tokens_list:
            sorted_unique_special_tokens_for_regex = sorted(list(set(self.special_tokens_list)), key=len, reverse=True)
            self.special_tokens_regex_pattern = f"({'|'.join(map(re.escape, sorted_unique_special_tokens_for_regex))})"
            self.special_tokens_regex: Optional[Pattern] = re.compile(self.special_tokens_regex_pattern)
        else:
            self.special_tokens_regex: Optional[Pattern] = None
    
    @classmethod
    def from_files(cls, 
                   vocab_filepath: str, 
                   merges_filepath: str, 
                   special_tokens: Optional[List[str]] = None) -> 'Tokenizer':
        """Loads a Tokenizer from vocabulary and merges files.

        Args:
            vocab_filepath (str): Path to the JSON file containing the vocabulary.
            merges_filepath (str): Path to the JSON file containing the merge rules.
            special_tokens (Optional[List[str]]): A list of special token strings.

        Returns:
            Tokenizer: An instance of the Tokenizer class initialized with the loaded data.
        
        Raises:
            FileNotFoundError: If vocab_filepath or merges_filepath does not exist.
            json.JSONDecodeError: If the files are not valid JSON.
            TypeError: If the loaded data is not in the expected format.
        """
        loaded_vocab: Dict[int, bytes]
        loaded_merges: List[Tuple[bytes, bytes]]
        
        if 'load_tokenizer_components' in globals() and callable(globals()['load_tokenizer_components']):
            loaded_vocab, loaded_merges = globals()['load_tokenizer_components'](vocab_filepath, merges_filepath)
        else:
            try:
                with open(vocab_filepath, 'r', encoding='utf-8') as f:
                    vocab_json = json.load(f)
                loaded_vocab = {int(k): base64.b64decode(v_str) for k, v_str in vocab_json.items()}

                with open(merges_filepath, 'r', encoding='utf-8') as f:
                    merges_json = json.load(f)
                loaded_merges = [(base64.b64decode(p1_str), base64.b64decode(p2_str)) for p1_str, p2_str in merges_json]
            except FileNotFoundError as e:
                print(f"Error: One or both tokenizer component files not found.\nVocab: {vocab_filepath}\nMerges: {merges_filepath}")
                raise e
            except Exception as e:
                print(f"Error loading tokenizer components directly from files: {e}")
                raise e
        
        return cls(loaded_vocab, loaded_merges, special_tokens)

    def _encode_bytes_with_merges(self, piece_bytes: bytes) -> List[int]:
        """Encodes a piece of bytes (typically a pre-tokenized word) using BPE merges.

        Args:
            piece_bytes (bytes): The byte sequence of a pre-tokenized word.

        Returns:
            List[int]: A list of token IDs representing the encoded piece.
        """
        if not piece_bytes: 
            return []
        
        tokens_as_bytes_list: List[bytes] = [bytes([b]) for b in piece_bytes]

        while len(tokens_as_bytes_list) > 1:
            best_pair_info: Optional[Tuple[int, int]] = None 
            
            for i in range(len(tokens_as_bytes_list) - 1):
                pair = (tokens_as_bytes_list[i], tokens_as_bytes_list[i+1])
                if pair in self.merges_map:
                    rank = self.merges_map[pair]
                    if best_pair_info is None or rank < best_pair_info[0]:
                        best_pair_info = (rank, i)
            
            if best_pair_info is None:
                break 

            _rank, idx = best_pair_info 
            merged_token_bytes = tokens_as_bytes_list[idx] + tokens_as_bytes_list[idx+1]
            tokens_as_bytes_list = tokens_as_bytes_list[:idx] + [merged_token_bytes] + tokens_as_bytes_list[idx+2:]
        
        final_ids: List[int] = []
        for b_sequence in tokens_as_bytes_list:
            if b_sequence in self.bytes_to_id:
                final_ids.append(self.bytes_to_id[b_sequence])
            else:
                # Fallback: if a multi-byte sequence is not found, encode its constituent bytes individually.
                # This handles cases where a sequence (e.g., a single byte from an unknown character after pre-tokenization)
                # was not part of any merge or the initial vocab explicitly, but its constituent bytes might be.
                # This is crucial for ensuring that any arbitrary byte sequence can be tokenized at least into its base bytes.
                # print(f"Warning: Byte sequence {b_sequence!r} not in bytes_to_id map. Attempting to break down.")
                for single_byte_val in b_sequence: 
                    single_byte_obj = bytes([single_byte_val])
                    if single_byte_obj in self.bytes_to_id:
                         final_ids.append(self.bytes_to_id[single_byte_obj])
                    else:
                        # This should ideally not happen if the base 256 bytes are always in vocab.
                        print(f"Critical Warning: Single byte {single_byte_obj!r} (value: {single_byte_val}) not in bytes_to_id map. Vocab may be malformed or incomplete.")
                        # Options: raise error, assign a specific UNK_ID, or skip. Skipping for now based on prior context.
                        pass 
        return final_ids
    
    def encode(self, text: str) -> List[int]:
        """Encodes a string of text into a list of token IDs.

        The encoding process involves:
        1. Splitting the text by special tokens (if any are defined and present).
        2. For segments that are special tokens, their pre-assigned IDs are used.
        3. For other segments, GPT-2's pre-tokenization regex (PAT_REGEX) is applied to get "words".
        4. Each "word" is converted to UTF-8 bytes.
        5. BPE merges are applied to these bytes via `_encode_bytes_with_merges`.
        6. All resulting token IDs are concatenated.

        Args:
            text (str): The input string to encode.

        Returns:
            List[int]: A list of token IDs representing the encoded text.
        """
        token_ids: List[int] = []
        
        if not text: # Handle empty string input immediately
            return []

        # Determine segments to process: either the whole text or parts split by special tokens
        segments_to_process: List[str]
        if not self.special_tokens_regex: # No special tokens defined or no regex compiled
            segments_to_process = [text]
        else:
            # Split text by special tokens, keeping the delimiters (special tokens themselves)
            # Example: text="a<EOS>b", regex capturing group for "<EOS>" -> parts=["a", "<EOS>", "b"]
            segments_to_process = self.special_tokens_regex.split(text)

        for segment in segments_to_process:
            if not segment: # re.split can produce empty strings if delimiters are adjacent or at ends
                continue

            if segment in self.special_token_to_id:
                # This segment is a known special token
                token_ids.append(self.special_token_to_id[segment])
            else:
                # This segment is not a special token, so pre-tokenize it using PAT_REGEX
                # and then apply BPE merges to each pre-token.
                for pre_token_match in self.PAT_REGEX.finditer(segment):
                    pre_token_str = pre_token_match.group(0)
                    pre_token_bytes = pre_token_str.encode("utf-8") # Convert pre-token to bytes
                    # Encode these bytes using BPE merges
                    token_ids.extend(self._encode_bytes_with_merges(pre_token_bytes))
        return token_ids

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        """Encodes an iterable of text strings, yielding token IDs.

        This method processes each string from the iterable independently and yields 
        its token IDs. It's suitable for tokenizing large collections of texts that 
        might not fit into memory if processed all at once as a single string.

        Args:
            iterable (Iterable[str]): An iterable (e.g., a list or generator) of strings to encode.

        Yields:
            int: Token IDs from the encoded text chunks.
        """
        for text_chunk in iterable:
            for token_id in self.encode(text_chunk):
                yield token_id
                
    def decode(self, ids: List[int]) -> str:
        """Decodes a list of token IDs back into a string.

        The decoding process involves:
        1. Converting each token ID to its byte sequence using the vocabulary.
           - Special token IDs are mapped back to their original string form (e.g., "<EOS>"), 
             then encoded to UTF-8 bytes.
           - Regular token IDs are mapped directly to their byte sequences from the vocab.
           - Unknown token IDs are skipped (as per project spec).
        2. Concatenating all byte sequences.
        3. Decoding the full byte sequence from UTF-8 to a string, using 'replace' for errors.

        Args:
            ids (List[int]): A list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        byte_chunks: List[bytes] = []
        for token_id in ids:
            if token_id in self.id_to_special_token:
                # Special tokens are stored as strings in id_to_special_token;
                # encode them back to bytes for consistent concatenation.
                byte_chunks.append(self.id_to_special_token[token_id].encode('utf-8'))
            elif token_id in self.vocab:
                byte_chunks.append(self.vocab[token_id])
            else:
                # Behavior for unknown ID: Project request implies skipping.
                # A warning could be logged here if desired for debugging.
                # print(f"Warning: Unknown token ID {token_id} encountered during decode. Skipping.")
                pass 
        
        full_byte_sequence = b"".join(byte_chunks)
        # Decode the full byte sequence, replacing any malformed UTF-8 sequences.
        return full_byte_sequence.decode("utf-8", errors="replace")

In [ ]:
#| hide
# Example to test methods including encode, encode_iterable, and decode
if __name__ == '__main__':
    print("--- Setting up Tokenizer for tests ---")

    vocab_for_tests = {
        **{i: bytes([i]) for i in range(256)},
        ord('h'): ord('h'), ord('e'): ord('e'), ord('l'): ord('l'), ord('o'): ord('o'), 
        ord(' '): ord(' '), ord('w'): ord('w'), ord('r'): ord('r'), ord('d'): ord('d'),
        ord('!'): ord('!'),
        256: b'\xe2\x82\xac', # Euro sign: €
        257: b'ell',
        258: b'ello',
        259: b' wo',
        260: b' wor',
        261: b'orld',
        262: b' world'
    }
    eos_bytes = b"<EOS>"
    pad_bytes = b"<PAD>"

    merges_for_tests = [
        (b'e', b'l'), (b'el', b'l'), (b'ell', b'o'), (b' ', b'w'), (b' w', b'o'), 
        (b' wo', b'r'), (b'o', b'r'), (b'or', b'l'), (b'orl', b'd'), (b' wor', b'ld')
    ]
    special_tokens_for_tests = ["<EOS>", "<PAD>"]
    
    tokenizer_for_tests = Tokenizer(vocab_for_tests.copy(), merges_for_tests, special_tokens_for_tests)
    
    # IDs for special tokens will be assigned by __init__
    eos_id = tokenizer_for_tests.special_token_to_id["<EOS>"]
    pad_id = tokenizer_for_tests.special_token_to_id["<PAD>"]
    print(f"Assigned <EOS> ID: {eos_id}, <PAD> ID: {pad_id}") # Should be 263, 264

    print("\n--- Testing decode method ---")
    decode_test_cases = {
        "simple sequence": ([ord('h'), 258, 262, ord('!')], "hello world!"),
        "sequence with special token": ([ord('h'), 258, eos_id, 262], "hello<EOS> world"),
        "empty list": ([], ""),
        "unknown token ID": ([ord('h'), 258, 999, 262], "hello world"), # 999 is unknown, should be skipped
        "only special tokens": ([eos_id, pad_id, eos_id], "<EOS><PAD><EOS>"),
        "unicode character from vocab": ([256], "€")
    }
    for name, (input_ids, expected_text) in decode_test_cases.items():
        try:
            result_text = tokenizer_for_tests.decode(input_ids)
            assert result_text == expected_text, f"Decode Test '{name}' FAILED. Expected '{expected_text}', Got '{result_text}' for IDs {input_ids}"
            print(f"Decode Test '{name}' PASSED.")
        except Exception as e:
            print(f"Decode Test '{name}' ERRORED for IDs {input_ids}: {e}")
            import traceback; traceback.print_exc()

    print("\n--- Testing encode/decode roundtrip ---")
    roundtrip_test_texts = [
        "hello world!", 
        "<EOS>hello<PAD> world€!", 
        "复杂的中文字符串", 
        "",
        "   leading and trailing spaces   ",
        "'s 'd 'll 've 're 'm 't",
        "A single byte like z then a special <EOS> token."
    ]
    for i, text in enumerate(roundtrip_test_texts):
        try:
            encoded_ids = tokenizer_for_tests.encode(text)
            decoded_text = tokenizer_for_tests.decode(encoded_ids)
            assert decoded_text == text, f"Roundtrip Test {i} FAILED for '{text}'. Encoded: {encoded_ids}, Decoded: '{decoded_text}'"
            print(f"Roundtrip Test {i} for '{text}' PASSED.")
        except Exception as e:
            print(f"Roundtrip Test {i} for '{text}' ERRORED: {e}")
            import traceback; traceback.print_exc()

    print("\n--- Testing decode with malformed UTF-8 ---")
    # Create a temporary tokenizer and add a malformed UTF-8 sequence to its vocab
    temp_vocab_malformed = tokenizer_for_tests.vocab.copy()
    malformed_id = 1000 
    temp_vocab_malformed[malformed_id] = b'\xff\xfe' # Invalid UTF-8 sequence
    tokenizer_malformed_test = Tokenizer(temp_vocab_malformed, merges_for_tests, special_tokens_for_tests)
    
    ids_with_malformed = [ord('h'), malformed_id, ord('i')]
    expected_malformed_decode = "h\ufffd\ufffdi" # Two replacement characters for b'\xff\xfe'
    decoded_malformed_text = tokenizer_malformed_test.decode(ids_with_malformed)
    assert decoded_malformed_text == expected_malformed_decode, \
        f"Decode malformed UTF-8 FAILED. Expected '{expected_malformed_decode}', Got '{decoded_malformed_text}'"
    print(f"Decode malformed UTF-8 PASSED. Decoded: '{decoded_malformed_text!r}'")

    print("\nAll Tokenizer class method tests (init, from_files, _encode_bytes, encode, encode_iterable, decode) completed.")
